In [7]:
import requests 
import re
import pandas as pd
import datetime

In [2]:
giscs = requests.get("https://www.wis-jma.go.jp/wcd/v1/json/accessibleMonitorJsonURL.json").json()
giscs = giscs.keys()
giscs

dict_keys(['GISC Beijing [CN]', 'GISC Brasilia [BR]', 'GISC Casablanca [MA]', 'GISC Exeter [GB]', 'GISC Melbourne [AU]', 'GISC Moscow [RU]', 'GISC Offenbach [DE]', 'GISC Seoul [KR]', 'GISC Tokyo [JP]', 'GISC Toulouse [FR]'])

In [3]:
def toURL(s):
    s = s.replace(" ","-")

    return s

giscs = [ toURL(g)  for g in giscs]
giscs

['GISC-Beijing-[CN]',
 'GISC-Brasilia-[BR]',
 'GISC-Casablanca-[MA]',
 'GISC-Exeter-[GB]',
 'GISC-Melbourne-[AU]',
 'GISC-Moscow-[RU]',
 'GISC-Offenbach-[DE]',
 'GISC-Seoul-[KR]',
 'GISC-Tokyo-[JP]',
 'GISC-Toulouse-[FR]']

In [4]:
def get_nc_dataseries(nc):
    m = re.search(r'\[(\w{2})\].txt',nc)
    
    if m:
        iso2 = m.group(1)
    else:
        tmp = nc.split("_")
        iso2 = tmp[-1].replace(".txt","")
    
    df = pd.read_csv(nc,header=None)
    df.columns = ["date","nr_data_and_products","size_data_products","nr_data_products_missing_md","size_data_products_missing_md","nr_metadata_unique_products_missing_md","nr_metadata_unique_products"]
    df["iso2"]=iso2
    
    return df
    
#get_nc_dataseries("https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Offenbach-[DE]_Kenya-[KE].txt")
#get_nc_dataseries("https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Exeter-[GB]_United-Kingdom.txt")

In [5]:
#https://www.wis-jma.go.jp/wcd/v1/json/GISC-Beijing-[CN]_cache.json
first = True
for g in giscs:
    
    url = "https://www.wis-jma.go.jp/wcd/v1/json/{gisc}_cache.json".format(gisc=g)
    r = requests.get(url)
    
    aor = r.json()
    
    for c in aor["centres"]:
        
        centre = c["centre"].replace(" ","-")
        
        #https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Offenbach-[DE]_Kenya-[KE].txt
        url2 = "https://www.wis-jma.go.jp/wcd/v1/json/timeplot_{gisc}_{nc}.txt".format(gisc=g,nc=centre)
        
        print(url2)
        tmp = get_nc_dataseries(url2)
        
        if first:
            df = tmp
        else:
            df = pd.concat([df,tmp])

        first=False

https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Beijing-[CN]_NC-NMC-[PK].txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Beijing-[CN]_NC-NMC-[CN].txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Beijing-[CN]_NC-NMC-[KP].txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Beijing-[CN]_NC-NMC-[HK].txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Beijing-[CN]_NC-WSO-[MO].txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Beijing-[CN]_NC-NMC-[MN].txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Beijing-[CN]_NC-NMC-[NP].txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Exeter-[GB]_United-Kingdom.txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Exeter-[GB]_Iceland.txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Exeter-[GB]_Tanzania.txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Exeter-[GB]_Netherlands.txt
https://www.wis-jma.go.jp/wcd/v1/json/timeplot_GISC-Exeter-[GB]_Ireland.txt
https://www.wis-jma.go.jp/wcd/v1/json/tim

In [11]:
df.to_csv("jma-nc-monitoring_{}.csv".format(datetime.datetime.now().date()),sep=";",index=False)